In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from collections import Counter
%matplotlib inline

In [2]:
#Считаем данные из файла
frame = pd.read_csv('data.csv', sep=',')

In [3]:
#Посмотрим на структуру данных и их количество
print frame.shape
print frame.info()
frame.head(10)

(4124734, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4124734 entries, 0 to 4124733
Data columns (total 6 columns):
event_id      object
date          object
cookie_id     object
session_id    object
event_type    object
domain_id     object
dtypes: object(6)
memory usage: 188.8+ MB
None


,event_id,date,cookie_id,session_id,event_type,domain_id
0,092656bc-da3a-42b2-87b2-bc7349cf1e91,2020-08-18 00:00:01.065710,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
1,4ee06cf5-24cc-478f-9acf-4e286516177f,2020-08-18 00:00:01.066429,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
2,ddc54ec1-46de-48ee-b906-9d8ce3d64b37,2020-08-18 00:00:02.582822,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
3,a34f5180-a533-4dd0-bb0c-a61bfb40b74a,2020-08-18 00:00:02.582836,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
4,962ae9a7-b768-46e2-b9bd-48dcdc78d0d9,2020-08-18 00:00:03.225834,b82ab5d5-228b-4f41-89c4-4eb74bfb4653,8645bfcd-c7be-4014-9f83-0c78be8d1f21,read_block ...,34308d56-1149-42a1-a0d9-5652c915ac9e
5,53accf77-822a-453b-afc4-f5d04b359680,2020-08-18 00:00:04.065633,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
6,2991abd9-129f-4c60-9411-be1bc3799aee,2020-08-18 00:00:04.066191,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
7,533497de-6e52-4328-a8e6-c0eaae26f2a5,2020-08-18 00:00:04.221122,b82ab5d5-228b-4f41-89c4-4eb74bfb4653,8645bfcd-c7be-4014-9f83-0c78be8d1f21,visit ...,34308d56-1149-42a1-a0d9-5652c915ac9e
8,19b790f4-fd77-492a-8c4b-77d22628b8e1,2020-08-18 00:00:05.567508,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
9,3f64504c-4e3a-434c-9e54-3e296b2ddc13,2020-08-18 00:00:05.568047,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block ...,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a


In [4]:
%%time
#Преобразуем столбец date в формат datetime
frame.date = frame.date.apply(pd.to_datetime)

CPU times: user 10min 56s, sys: 11.7 s, total: 11min 8s
Wall time: 11min 3s


In [5]:
#Удалим символы конца строки во всех ячейках с данными
for column in frame.columns.drop('date', 1):
    frame[column] = frame[column].apply(lambda s: s.strip())

In [6]:
#Посмотрим, для какого количества сайтов у нас имеются данные в исходном датасете
pd.unique(frame.domain_id).shape[0]

9

Видно, что сайтов не много. Учитывая, что они могут быть связаны между собой, не будем разделять датасет по каждому сайту и сформируем список признаков пользователей по совокупности сайтов. Это позволит при кластеризации учесть возможные взаимосвязи между предпочтениями пользователей и посещаемыми ими сайтами.

In [7]:
#Посмотрим, какие действия совершали пользователи на сайтах
pd.unique(frame.event_type).tolist()

['read_block',
 'visit',
 'change_resolution',
 'filter_mortgage_change',
 'click_scbform_button_callrequest',
 'click',
 'callback_exit',
 'hover',
 'Choose_flat',
 'filter_change',
 'choose_flat',
 'change_slider',
 'viber',
 'submit_flat_applicaiton_click',
 'callback_start',
 'chat_start',
 'whatsUp',
 'chat_exit',
 'clcik',
 'taxi_start',
 'taxi_exit',
 'choose_dream_flat_click_podrobnee']

Подумаем, какие признаки для пользователей можно выделить:
- num_sessions_for_domain_i (i = 1...9) - число сессий для сайта с domain_id = i;
- avg_session_duration_for_domain_i (i = 1...9) - средняя длительность сесcии для сайта с domain_id = i;
- num_actions_i_for_domain_j (i = 1...22, j = 1...9) - число действий с event_type = i на сайте с domain_id = j;
- avg_time_of_session_begining_for_domain_i (i = 1...9) - среднее время начала сессии для сайта с domain_id = i

Здесь i и j - условные обозначения для идентификаторов сайтов и типов действий.

In [8]:
# Поскольку исходная выборка слишком большая, можно было бы ее на несколько частей и провести кластеризацию для каждой из них по-отдельности
import random
cookies_id_list = pd.unique(frame.cookie_id)

# Перемешаем пользователей
random.shuffle(cookies_id_list)

# Разобьем список идентификаторов пользователей на 20 частей
num_items = len(cookies_id_list) // 20
remain_items = len(cookies_id_list) % 20
print num_items, remain_items

# Последние 2 объекта можно отбросить
# Видно, что в каждую выборку попадет более 8000 пользователей.
# Это достаточно большое число объектов при условии, что признаков у нас будет порядка 100

8356 2


In [13]:
# Для ускорения процесса и для примера проведем кластеризацию для первых 100000 объектов из иходного датасета
frame_in = frame.iloc[:100000]
frame_in.head(10)

,event_id,date,cookie_id,session_id,event_type,domain_id
0,092656bc-da3a-42b2-87b2-bc7349cf1e91,2020-08-18 00:00:01.065710,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
1,4ee06cf5-24cc-478f-9acf-4e286516177f,2020-08-18 00:00:01.066429,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
2,ddc54ec1-46de-48ee-b906-9d8ce3d64b37,2020-08-18 00:00:02.582822,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
3,a34f5180-a533-4dd0-bb0c-a61bfb40b74a,2020-08-18 00:00:02.582836,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
4,962ae9a7-b768-46e2-b9bd-48dcdc78d0d9,2020-08-18 00:00:03.225834,b82ab5d5-228b-4f41-89c4-4eb74bfb4653,8645bfcd-c7be-4014-9f83-0c78be8d1f21,read_block,34308d56-1149-42a1-a0d9-5652c915ac9e
5,53accf77-822a-453b-afc4-f5d04b359680,2020-08-18 00:00:04.065633,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
6,2991abd9-129f-4c60-9411-be1bc3799aee,2020-08-18 00:00:04.066191,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
7,533497de-6e52-4328-a8e6-c0eaae26f2a5,2020-08-18 00:00:04.221122,b82ab5d5-228b-4f41-89c4-4eb74bfb4653,8645bfcd-c7be-4014-9f83-0c78be8d1f21,visit,34308d56-1149-42a1-a0d9-5652c915ac9e
8,19b790f4-fd77-492a-8c4b-77d22628b8e1,2020-08-18 00:00:05.567508,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a
9,3f64504c-4e3a-434c-9e54-3e296b2ddc13,2020-08-18 00:00:05.568047,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,f6ffb2a7-dfbd-40d7-ad25-a08c5a3a1323,read_block,a1fb2d2a-9c3e-49e2-a651-cf52fbf0212a


Сформируем выборку, в которую включим описанные выше признаки

In [14]:

cookie_id_list = pd.unique(frame_in.cookie_id)
domain_id_list = pd.unique(frame_in.domain_id)
event_types_list = pd.unique(frame_in.event_type)
data = pd.DataFrame()

#Создадим шаблон выборки
data['cookie_id'] = cookie_id_list
for i in range(len(domain_id_list)):
    data['num_sessions_for_domain_{}'.format(i)] = np.zeros(len(cookie_id_list), dtype=int)
    data['avg_session_duration_for_domain_{}'.format(i)] = np.zeros(len(cookie_id_list), dtype=int)
    data['avg_time_of_session_begining_for_domain_{}'.format(i)] = np.zeros(len(cookie_id_list), dtype=int)
    for j in range(len(event_types_list)):
        data['num_actions_{0}_for_domain_{1}'.format(j, i)] = np.zeros(len(cookie_id_list), dtype=int)


#Заполним шаблон выборки
#Переберем всех пользователей
for i, cookie_id in enumerate(cookie_id_list):

    #Возьмем срез датафрейма, соответствующий выбранному пользователю
    frame_for_user = frame_in.loc[frame_in.cookie_id == cookie_id]

    #Переберем все сайты для каждого пользователя
    for j, domain_id in enumerate(domain_id_list):

        #Возьмем срез, соответствующий выбранному сайту
        frame_for_user_for_domain = frame_for_user.loc[frame_for_user.domain_id == domain_id]

        # num_sessions_for_domain_j

        #Подсчитаем число сессий для соответствующего сайта
        session_id_list = pd.unique(frame_for_user_for_domain.session_id)
        num_sessions = len(session_id_list)

        #Добавим найденное значение в датафрейм
        data.at[i, 'num_sessions_for_domain_{}'.format(j)] = num_sessions

        # num_actions_k_for_domain_j 

        #Подсчитаем число действий каждого типа для данного сайта
        event_types_list_for_domain = frame_in.loc[(frame_in['cookie_id'] == cookie_id) & \
                                          (frame_in['domain_id'] == domain_id)]['event_type']
        event_types_counter = Counter(event_types_list_for_domain)

        #Запишем найденные значения числа действий каждого типа в выборку
        for k, event_type in enumerate(event_types_list):
            if event_type in event_types_counter.keys():
                data.at[i, 'num_actions_{0}_for_domain_{1}'.format(k, j)] = event_types_counter[event_type]

        # avg_session_duration_for_domain_j 
        # и
        # avg_daytime_of_session_begining_for_domain_j

        #Создадим массив для подсчета средней длительности сессии для сайта
        sessions_duration_list = []

        #Создадим массив для подсчета среднего времеени начала сессии для сайта
        session_beginings_list = []

        #Проверим, были ли сессии для данного сайта у данного пользователя, чтобы посчитать среднюю длительность сессии 
        if num_sessions != 0:

            #Переберем все сессии для данного сайта
            for session_id in session_id_list:

                #Подсчитаем длительность сессии
                begining = frame_in.loc[(frame_in['cookie_id'] == cookie_id) & \
                                        (frame_in['session_id'] == session_id)]['date'].tolist()[0]
                ending = frame_in.loc[(frame_in['cookie_id'] == cookie_id) & \
                                    (frame_in['session_id'] == session_id)]['date'].tolist()[-1]
                duration = ending - begining

                session_beginings_list.append(begining.hour)
                sessions_duration_list.append(duration.seconds)

            #Добавим полученное значение длительности сессии в выборку
            data.at[i, 'avg_session_duration_for_domain_{}'.format(j)] = np.array(sessions_duration_list).mean()

            #Добавим среднее время начала сессии (в часах) в датафрейм
            data.at[i, 'avg_time_of_session_begining_for_domain_{}'.format(j)] = np.array(session_beginings_list).mean()


        else:
            data.at[i, 'avg_session_duration_for_domain_{}'.format(j)] = 0

data

,cookie_id,num_sessions_for_domain_0,avg_session_duration_for_domain_0,avg_time_of_session_begining_for_domain_0,num_actions_0_for_domain_0,num_actions_1_for_domain_0,num_actions_2_for_domain_0,num_actions_3_for_domain_0,num_actions_4_for_domain_0,num_actions_5_for_domain_0,...,num_actions_8_for_domain_8,num_actions_9_for_domain_8,num_actions_10_for_domain_8,num_actions_11_for_domain_8,num_actions_12_for_domain_8,num_actions_13_for_domain_8,num_actions_14_for_domain_8,num_actions_15_for_domain_8,num_actions_16_for_domain_8,num_actions_17_for_domain_8
0,9e2c1cf7-83a4-4fcf-9d87-63e316a0c91f,1,38895,0,51559,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,b82ab5d5-228b-4f41-89c4-4eb74bfb4653,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,fd9b95c0-5c0d-4cb7-a966-2f38bd8ff836,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,b2b702a6-2c1e-488e-8c57-6e06b8f9f1e4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,852a6954-de45-432e-bd13-11de49ba0a68,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,91dd4eb3-90ba-4cc8-842f-1aefe82d131a,3,7924,4,0,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,a807399c-5af5-4c59-9fef-43c0557361b7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,e560b1fd-3c03-48a7-a206-6ecdf8dd8125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,c1a47c6e-69b4-40d8-9e69-1af8bba385f6,1,18144,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,34cb824f-0d4a-49e6-8417-bfe5eec8b176,3,9534,3,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Отбросим идентификааторы пользователей из выборки (их нам надо кластеризовать)
X = data.drop('cookie_id', 1)

# Кластеризация

Для кластеризации выберем алгоритм DBSCAN, так как он:
- применим для больших выборок и умеренного количестваа кластеров (вряд ли при текущей постановке задачи необходимо будет получить большое число кластеров для пользователей);
- способен отсеивать выбросы (которых, скорее всего, много, так как многие пользователи могут являться случайными);
- применим для невыпуклых и криволинейных кластеров.

In [16]:
from sklearn.cluster import DBSCAN

# Будем использовать алгоритм со стандартными параметрами,
# так как для оптимального подбора параметров необходимо проводить более комплексный анализ данных,
# который требует значительно больше времени

model = DBSCAN()
labels = model.fit_predict(X)

# Посмотрим, сколько кластеров мы получили
print 'Число кластеров: ', len(np.unique(labels))

# Сопоставим полученные метки кластеров с идентификаторы пользователей, которые необходимо было кластеризовать
clustering_results = pd.DataFrame()
clustering_results['cookie_id'] = data['cookie_id']
clustering_results['cluster_label'] = labels

# Выведем полученный результат
Counter(clustering_results['cluster_label'])

Число кластеров:  7


Counter({-1: 3005, 0: 5, 1: 6, 2: 7, 3: 6, 4: 6, 5: 7})